<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
class Layer():
  def __init__(self, name, units, input_dim):
    self.name = name
    self.w = [[random.uniform(-2,2) for _ in range(input_dim)] for _ in range(units)]
    self.b = [0 for _ in range(units)]
    self.output = [bias for bias in self.b]
  def call(self, inputs):
    self.output = [bias for bias in self.b]
    for row_index, col in range(len(self.w)):
      for col_index, weight in range(len(col)):
        self.output[row_index] += weight * inputs[col_index]
      self.output[row_index] = 1.0 if self.output[row_index] > 0.0 else 0.0
    return self.output 
  def update_weights(expected_output, inputs):
    for row_index, col in range(len(self.w)):
      for col_index, weight in range(len(col)):
        error = self.output[row_index] - expected_output
        self.w[row_index][col_index] = self.w[row_index][col_index] - (error * inputs[row_index])
    

In [47]:
from pprint import pprint
layer1 = Layer("Hidden1", 3, 3)
layer2 = Layer("Output", 3, 3)

layers = [layer1, layer2]
pprint(vars(layer1))
pprint(vars(layer2))

{'b': [0, 0, 0],
 'name': 'Hidden1',
 'output': [0, 0, 0],
 'w': [[1.8640948969639464, -1.9070156691436115, 1.2860123652997104],
       [0.49246780554550673, 0.5196057972898176, -1.3236709193863039],
       [-1.699439722368345, -0.3972276167550248, 0.0798427851818877]]}
{'b': [0, 0, 0],
 'name': 'Output',
 'output': [0, 0, 0],
 'w': [[1.5038643176179538, -1.338909034954138, 0.06958006183710319],
       [-0.14830082005529688, -0.359288338452326, 0.31937482819107554],
       [0.7885429653733333, -0.2242564955695725, -0.9351073576933877]]}


In [4]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

In [5]:
iris = load_iris()

iris_columns = ['sepal_len', 'sepal_width', 'petal_length', 'petal_width', 'target']
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris_columns)

In [31]:
from sklearn.model_selection import train_test_split
X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
y = df[['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)

X_train.reset_index(level=0, inplace=True)
y_train.reset_index(level=0, inplace=True)

In [35]:
import random
random.seed(34)
selected_rows = random.sample(range(0,119),   5)

In [36]:
X_train.loc[selected_rows, :]

,index,sepal_len,sepal_width,petal_length,petal_width
67,80,5.5,2.4,3.8,1.1
45,133,6.3,2.8,5.1,1.5
74,11,4.8,3.4,1.6,0.2
3,114,5.8,2.8,5.1,2.4
107,16,5.4,3.9,1.3,0.4


In [37]:
def train_weights_using_mini_batch_gradient_descent(X_train, y_train, num_epochs, learning_rate, layers):
  # weights for all the features 
  index = 0;
  df_plot = []
  while index < num_epochs:
    mini_batch = X_train.loc[selected_rows, :]
    y_train = y_train.loc[selected_rows, :]
    for rowIndex, row in mini_batch.iterrows():
      for layer in layers:
        layer(row)
    bias = bias + learning_rate * (mse/mini_batch.shape[0])
    weights = [weight + learning_rate * (total_gradient[weight_index]/mini_batch.shape[0]) for weight_index, weight in enumerate(weights)]
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (index, learning_rate, mse/mini_batch.shape[0]))
    df_plot.append({mse/mini_batch.shape[0], index})
    index = index + 1
  df_plot = pd.DataFrame(df_plot, columns=['mse', 'epoch'])
  return weights, bias, df_plot


print("Creating model using training data")
(weights_using_mini_batch_gradient_descent_train, bias_using_mini_batch_gradient_descent_train, df_plot_train) = \
  train_weights_using_mini_batch_gradient_descent(X_train, y_train, 10, 0.1, layers)
print(weights_using_mini_batch_gradient_descent_train)

Creating model using training data
>epoch=0, lrate=0.100, error=1.800
>epoch=1, lrate=0.100, error=0.800
>epoch=2, lrate=0.100, error=0.800
>epoch=3, lrate=0.100, error=0.800
>epoch=4, lrate=0.100, error=0.800
>epoch=5, lrate=0.100, error=0.800
>epoch=6, lrate=0.100, error=0.800
>epoch=7, lrate=0.100, error=0.800
>epoch=8, lrate=0.100, error=0.800
>epoch=9, lrate=0.100, error=0.800
[56.64002778469252, 1.0805608661440433, -0.006592187992660023, 2.0026005541560683]
